In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import timedelta
path = r'D:\DATA\ENTSO-E\SFTP_Germany\Production per unit\Fossil Gas'

In [3]:
folder_path = r"D:\DATA\Epex\french_aggregated_bids"

# Initialize an empty dictionary to store DataFrames
dataframes_dict = {}

# Iterate through files in the directory
for filename in os.listdir(folder_path):
    print(filename)
    filepath = os.path.join(folder_path, filename)
    dataframe_name = os.path.splitext(filename)[0]
    df = pd.read_excel(filepath, skiprows=1)
    dataframes_dict[dataframe_name] = df

# Display the keys of the dictionary to verify the names
print(dataframes_dict.keys())

# Optional: Display the first few rows of each DataFrame to verify the content
for key, df in dataframes_dict.items():
    print(f"DataFrame for {key}:")
    print(df.head())

auction_aggregated_curves_france_20160101.xlsx
auction_aggregated_curves_france_20160102.xlsx
auction_aggregated_curves_france_20160103.xlsx
auction_aggregated_curves_france_20160104.xlsx
auction_aggregated_curves_france_20160105.xlsx
auction_aggregated_curves_france_20160106.xlsx
auction_aggregated_curves_france_20160107.xlsx
auction_aggregated_curves_france_20160108.xlsx
auction_aggregated_curves_france_20160109.xlsx
auction_aggregated_curves_france_20160110.xlsx
auction_aggregated_curves_france_20160111.xlsx
auction_aggregated_curves_france_20160112.xlsx
auction_aggregated_curves_france_20160113.xlsx
auction_aggregated_curves_france_20160114.xlsx
auction_aggregated_curves_france_20160115.xlsx
auction_aggregated_curves_france_20160116.xlsx
auction_aggregated_curves_france_20160117.xlsx
auction_aggregated_curves_france_20160118.xlsx
auction_aggregated_curves_france_20160119.xlsx
auction_aggregated_curves_france_20160120.xlsx
auction_aggregated_curves_france_20160121.xlsx
auction_aggre

OptionError: "No such keys(s): 'io.excel.zip.reader'"

In [ ]:
# Initialize an empty dictionary to store DataFrames
dataframes_dict = {}

# Iterate through files in the directory
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        # Read each CSV file and store it in the dictionary with the file name (without extension) as the key
        filepath = os.path.join(path, filename)
        dataframe_name = os.path.splitext(filename)[0]  # Remove the ".csv" extension
        dataframes_dict[dataframe_name] = pd.read_csv(filepath, index_col=0, parse_dates=True, sep="\t")

for key, dataframe in dataframes_dict.items():
    dataframe['Price'] = dataframe['Price'].interpolate(method='linear')
    dataframe['ShareOffered'] = dataframe['ActualGenerationOutput'] / dataframe['InstalledGenCapacity']

dataframes_dict['11W0-0000-0103-9']

In [ ]:
# Create a new dictionary to store the averaged data
sub_data = {}
candidats = 0

# Iterate through dataframes_dict and process each DataFrame
for key, dataframe in dataframes_dict.items():
    dataframe = dataframe[(dataframe['ShareOffered'] > 0.2) & (dataframe['ShareOffered'] < 0.95) & (dataframe['Price'] > 0)]

    if dataframe.empty:
        continue

    filtered_dataframe = dataframe[['ShareOffered', 'ActualGenerationOutput', 'Price']]
    daily_average_dataframe = filtered_dataframe.resample('h').mean()
    sub_data[key] = daily_average_dataframe

    # plt.figure(figsize=(10, 6))
    # plt.scatter(dataframe['ShareOffered'], dataframe['Price'], alpha=0.5, label='Data')

    # Linear regression model
    X = dataframe['ShareOffered'].values.reshape(-1, 1)
    y = dataframe['Price'].values
    model = LinearRegression()
    model.fit(X, y)

    if model.coef_[0] < 1:
        continue

    if model.score(X, y) < 0.01:
        continue

    predicted_prices = model.predict(X)

    plt.plot(dataframe['ShareOffered'], predicted_prices, color='red', linewidth=2, label='Linear Model Fit')
    plt.title(f'ShareOffered vs. Price ({key})')
    plt.xlabel('ShareOffered')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f'Linear Model Parameters for {key}:')
    print('Slope (Coefficient):', model.coef_[0])
    print('Intercept:', model.intercept_)
    print('R-squared:', model.score(X, y))
    print('-' * 40)

    candidats += 1

print("Number of power plants reactive to price: ", candidats)

In [ ]:
dataframes_dict['11WD7BISC5GKW3-T']

In [ ]:
centrale = "11WD7BISC5GKW3-T"

dataframe = dataframes_dict[centrale]
dataframe = dataframe.loc[dataframe.index.year <= 2020]
filtered_dataframe = dataframe[(dataframe['Price'] >= 0)][['ShareOffered', 'ActualGenerationOutput', 'Price']]
daily_average_dataframe = filtered_dataframe.resample('H').mean()
print(dataframe)
plt.figure(figsize=(10, 6))
plt.scatter(daily_average_dataframe['ShareOffered'], daily_average_dataframe['Price'], alpha=0.5, label='Data')
plt.title(f'Offered production as fraction of capacity vs. Price '+ centrale)
plt.xlabel('Share of capacity (%)')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


representative_day = dataframes_dict[centrale].loc['2018-12-01':'2018-12-08'].sort_index()

 #Random date

# Create a new figure with two y-axes sharing the same x-axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the Price curve on the left y-axis
ax1.plot(representative_day['Price'], label='Price', color='blue')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Create a twin Axes sharing the xaxis
ax2 = ax1.twinx()

# Plot the ShareOffered curve on the right y-axis
ax2.plot(representative_day['ShareOffered'], label='ShareOffered', color='green')
ax2.set_ylabel('ShareOffered', color='green')
ax2.tick_params(axis='y', labelcolor='green')

# Show legend for both curves
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Set the title
plt.title('Price and ShareOffered' + centrale + ' 1-8 December 2018')

# Show the plot
plt.show()

In [ ]:
daily_average_dataframe['ShareOffered']